In [3]:
import os
os.sys.path.append('..')

In [4]:
import warnings
warnings.simplefilter("ignore")

# autoreload import your package
%load_ext autoreload
%autoreload 2

In [9]:
from loguru import logger
logger.remove()
logger.add(os.sys.stdout, level="ERROR", colorize=True, format="<level>{time} | {message}</level>")
# import_dir(ta_dir, verbose=False)
warnings.simplefilter("ignore")

# Args

In [23]:
from run import set_seed, get_args, Exp_Main

In [51]:
# mimic cli args to avoid code duplication
argv = """python -u run.py \
  --root_path ../dataset/stocks/ \
  --data_path OXY_2019.csv.gz \
  --checkpoints ./checkpoints/ \
  --model_id Exchange \
  --model ETSformer \
  --data custom \
  --features M \
  --seq_len 128 \
  --pred_len 48 \
  --e_layers 2 \
  --d_layers 2 \
  --enc_in 12 \
  --dec_in 12 \
  --c_out 12 \
  --des 'Exp' \
  --K 0 \
  --learning_rate 1e-3 \
  --target RSMKs_18_144_72_2ref_2ref \
  --itr 1
"""
argv = argv.replace("\\n", "").split()[3:]
args = get_args(argv)
args

Args in experiment:
Namespace(K=0, activation='sigmoid', batch_size=32, c_out=12, checkpoints='./checkpoints/', d_ff=2048, d_layers=2, d_model=512, damping_learning_rate=0, data='custom', data_path='OXY_2019.csv.gz', dec_in=12, des="'Exp'", devices='0,1,2,3', dropout=0.2, e_layers=2, embed='timeF', enc_in=12, features='M', freq='h', gpu=0, itr=1, label_len=0, learning_rate=0.001, lradj='exponential_with_warmup', min_lr=1e-30, model='ETSformer', model_id='Exchange', n_heads=8, num_workers=10, optim='adam', output_attention=False, patience=5, pred_len=48, root_path='../dataset/stocks/', seq_len=128, smoothing_learning_rate=0, std=0.2, target='RSMKs_18_144_72_2ref_2ref', train_epochs=15, use_gpu=True, use_multi_gpu=False, warmup_epochs=3)


Namespace(K=0, activation='sigmoid', batch_size=32, c_out=12, checkpoints='./checkpoints/', d_ff=2048, d_layers=2, d_model=512, damping_learning_rate=0, data='custom', data_path='OXY_2019.csv.gz', dec_in=12, des="'Exp'", devices='0,1,2,3', dropout=0.2, e_layers=2, embed='timeF', enc_in=12, features='M', freq='h', gpu=0, itr=1, label_len=0, learning_rate=0.001, lradj='exponential_with_warmup', min_lr=1e-30, model='ETSformer', model_id='Exchange', n_heads=8, num_workers=10, optim='adam', output_attention=False, patience=5, pred_len=48, root_path='../dataset/stocks/', seq_len=128, smoothing_learning_rate=0, std=0.2, target='RSMKs_18_144_72_2ref_2ref', train_epochs=15, use_gpu=True, use_multi_gpu=False, warmup_epochs=3)

In [ ]:

Exp = Exp_Main

for ii in range(args.itr):
    print(ii)
    set_seed(ii)
    # setting record of experiments
    setting = '{}_{}_{}_ft{}_sl{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_K{}_lr{}_{}_{}'.format(
        args.model_id,
        args.model,
        args.data,
        args.features,
        args.seq_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.K,
        args.learning_rate,
        args.des, ii)

#     if os.path.exists(os.path.join(args.checkpoints, setting)):
#         print('skipping exists')
#         continue

    exp = Exp(args)  # set experiments
    print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
    exp.train(setting)

    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting, data='val')
    exp.test(setting, data='test')

    torch.cuda.empty_cache()


0
Use GPU: cuda:0
>>>>>>>start training : Exchange_ETSformer_custom_ftM_sl128_pl48_dm512_nh8_el2_dl2_df2048_K0_lr0.001_'Exp'_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 37203
val 5294
test 10632
	iters: 100, epoch: 1 | loss: 0.4932106
	speed: 0.0565s/iter; left time: 978.9201s
	iters: 200, epoch: 1 | loss: 0.4972929
	speed: 0.0485s/iter; left time: 835.0221s
	iters: 300, epoch: 1 | loss: 0.4357589
	speed: 0.0476s/iter; left time: 815.1622s
	iters: 400, epoch: 1 | loss: 0.5983973
	speed: 0.0477s/iter; left time: 811.9901s
	iters: 500, epoch: 1 | loss: 0.6287826
	speed: 0.0484s/iter; left time: 820.1333s
	iters: 600, epoch: 1 | loss: 0.4684563
	speed: 0.0488s/iter; left time: 821.1888s
	iters: 700, epoch: 1 | loss: 1.0543735
	speed: 0.0484s/iter; left time: 810.2026s
	iters: 800, epoch: 1 | loss: 0.7420598
	speed: 0.0474s/iter; left time: 788.7177s
	iters: 900, epoch: 1 | loss: 0.5108218
	speed: 0.0474s/iter; left time: 784.3583s
	iters: 1000, epoch: 1 | loss: 0.5239242
	speed: 0.0505s/iter; left 

In [49]:
%debug

> /media/wassname/SGIronWolf/projects5/investing/ETSformer/models/etsformer/encoder.py(155)forward()
    153         growth = growth.view(b, t, self.c_out, 1)
    154         season = season.view(b, t, self.c_out, 1)
--> 155         level = level.view(b, t, self.c_out, 1)
    156         out = self.es(level - season, aux_values=growth)
    157         out = rearrange(out, 'b t h d -> b t (h d)')

ipdb> q
